In [7]:
import cv2
import hyperspy.api as hs
import kornia.feature as KF
import kornia as K
import numpy as np
import torch
import matplotlib.pyplot as plt
from kornia_moons.viz import *
from kornia_moons.feature import *
%matplotlib qt5

In [8]:
device = torch.device("cpu")
feature = KF.KeyNetAffNetHardNet(5000, True).eval().to(device)

In [9]:
def numpy_to_torch_tensor(npy):
    npy = npy - np.amin(npy)
    npy = 255*(npy/np.amax(npy))
    tensor = np.zeros((1,1,np.shape(npy)[0],np.shape(npy)[1]))
    tensor[0,0] = npy
    tensor = torch.FloatTensor(tensor)
    return tensor

In [10]:
img0 = numpy_to_torch_tensor(np.load(r"C:\Users\tas72\Documents\PhD\dg606\20230624\Stitching_and_corr\test\20230624_100334_vbf.npy"))
img1 = numpy_to_torch_tensor(np.load(r"C:\Users\tas72\Documents\PhD\dg606\20230624\Stitching_and_corr\test\20230624_100824_vbf.npy"))
input_dict = {"image0":img0,
             "image1":img1}

In [15]:
feature = KF.KeyNetAffNetHardNet(5000, True).eval().to(device)
adalam_config = {"device": device}
hw1 = torch.tensor(img1.shape[2:])
hw2 = torch.tensor(img1.shape[2:])


In [16]:
with torch.inference_mode():
    lafs1, resps1, descs1 = feature(img0)
    lafs2, resps2, descs2 = feature(img1)
    dists, idxs = KF.match_adalam(
        descs1.squeeze(0),
        descs2.squeeze(0),
        lafs1,
        lafs2,  # Adalam takes into account also geometric information
        config=adalam_config,
        hw1=hw1,
        hw2=hw2,  # Adalam also benefits from knowing image size
    )

In [17]:
print(f"{idxs.shape[0]} tentative matches with AdaLAM")

461 tentative matches with AdaLAM


In [18]:
def get_matching_keypoints(lafs1, lafs2, idxs):
    mkpts1 = KF.get_laf_center(lafs1).squeeze()[idxs[:, 0]].detach().cpu().numpy()
    mkpts2 = KF.get_laf_center(lafs2).squeeze()[idxs[:, 1]].detach().cpu().numpy()
    return mkpts1, mkpts2


In [19]:
mkpts1, mkpts2 = get_matching_keypoints(lafs1, lafs2, idxs)

Fm, inliers = cv2.findFundamentalMat(mkpts1, mkpts2, cv2.USAC_MAGSAC, 0.75, 0.999, 100000)
inliers = inliers > 0
print(f"{inliers.sum()} inliers with AdaLAM")

268 inliers with AdaLAM


In [31]:
Tm,inliners = cv2.estimateAffinePartial2D(mkpts1,mkpts2,confidence= 0.999,ransacReprojThreshold=0.99,refineIters=100000)

In [32]:
print(f"{inliers.sum()} inliers with AdaLAM")

268 inliers with AdaLAM


In [33]:
Tm

array([[ 1.00043958e+00,  6.00119099e-04, -3.84868873e+01],
       [-6.00119099e-04,  1.00043958e+00,  3.28275265e+02]])

In [26]:
draw_LAF_matches(
    lafs1,
    lafs2,
    idxs,
    K.tensor_to_image(img0),
    K.tensor_to_image(img1),
    inliers,
    draw_dict={"inlier_color": (0.2, 1, 0.2), "tentative_color": (1, 1, 0.2, 0.3), "feature_color": None, "vertical": False},
)

array([[-2.98204032e-09, -6.10998475e-05, -1.25948367e-03],
       [ 6.16093288e-05,  5.60208921e-07, -1.66586067e-02],
       [-1.89629048e-02,  1.39584277e-02,  5.41887185e+00]])